위에 텍스트 있는 사진 찾기

In [8]:
import easyocr
import glob
import cv2
import numpy as np
import os

reader=easyocr.Reader(['en'],gpu=True)

image_path='./data/data_contrast_before/Image/'


cls_names=['Others','Porosity','Slag']

uptext_exists=[]

for cls_name in cls_names:
    
    image_paths=glob.glob(image_path + cls_name+'/*.jpg')

    
    for img_path in image_paths:
        
        results=reader.readtext(img_path)
        
        if not results:
            continue
        
        min_y = 99999  # 임의의 큰 수로 초기화
        threshold = 90
        text_found = False

        for detection in results:
            top_left = tuple(map(int, detection[0][0]))
            if top_left[1] < min_y and top_left[1] > threshold:
                min_y = top_left[1]
                text_found = True
        
        if text_found:
            print(f'{img_path}에 텍스트 있음')
            uptext_exists.append(img_path)
            

./data/data_contrast_before/Image/Others/6054-003175-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Others/6054-040837-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Others/j121-2.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-000529-A1-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-002716-A1-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-003175-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-003177-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-003177-2.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-020796-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-035966-R1-1-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-038086-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/6054-039680-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Porosity/h122-1.jpg에 텍스트 있음
./data/data_contrast_before/Image/Slag/s13-2.jpg에 텍스트 있음
./data/data_contrast_before/Image/Slag/s13-3.jpg에 텍스트 있

In [9]:
len(uptext_exists)

15

In [10]:
uptext_exists

['./data/data_contrast_before/Image/Others/6054-003175-1.jpg',
 './data/data_contrast_before/Image/Others/6054-040837-1.jpg',
 './data/data_contrast_before/Image/Others/j121-2.jpg',
 './data/data_contrast_before/Image/Porosity/6054-000529-A1-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-002716-A1-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-003175-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-003177-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-003177-2.jpg',
 './data/data_contrast_before/Image/Porosity/6054-020796-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-035966-R1-1-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-038086-1.jpg',
 './data/data_contrast_before/Image/Porosity/6054-039680-1.jpg',
 './data/data_contrast_before/Image/Porosity/h122-1.jpg',
 './data/data_contrast_before/Image/Slag/s13-2.jpg',
 './data/data_contrast_before/Image/Slag/s13-3.jpg']

Annotation_merged 만들기

In [62]:
import pandas as pd
from pathlib import Path
import os

image_path='./data/data_contrast_before/Image/'

cls_names=['Others','Porosity','Slag']
total_paths=[]

for cls_name in cls_names:
    
    image_paths=glob.glob(image_path + cls_name+'/*.jpg')
    total_paths.extend(image_paths)

df=pd.DataFrame(columns=['dataset','image_number','image_name','bndbox','labels','img_shape'])
print(f'텍스트 제외 전 :{len(total_paths)}')

텍스트 제외 전 :2289


In [63]:
total_paths=[path for path in total_paths if path not in uptext_exists]
print(f'텍스트 있는 사진: {len(uptext_exists)}, 텍스트 제외 사진 : {len(total_paths)}')

텍스트 있는 사진: 15, 텍스트 제외 사진 : 2274


In [64]:
df['dataset']=[str(path).split('/')[-2] for path in total_paths]

df['image_number']=list(range(1,len(total_paths)+1))

df['image_name']=[os.path.basename(str(path)) for path in total_paths]



In [65]:
df.head()

,dataset,image_number,image_name,bndbox,labels,img_shape
0,Others,1,6054-000130-1.jpg,NaN,NaN,NaN
1,Others,2,6054-000130-1-2.jpg,NaN,NaN,NaN
2,Others,3,6054-000134-1-1.jpg,NaN,NaN,NaN
3,Others,4,6054-000134-1-2.jpg,NaN,NaN,NaN
4,Others,5,6054-001180-A1-1.jpg,NaN,NaN,NaN


In [66]:
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree

bndbox_list=[]
imgshape_list=[]
label_list=[]

for path in total_paths:
    xml_path=str(path).replace('Image','Label').replace('jpg','xml')
    img_size=[]
    bboxes=[]
    labels=[]
    with open(xml_path,'r') as xml:
        tree=Et.parse(xml)
        root=tree.getroot()
        
        objects=root.findall('object')
        sizes=root.findall('size')
        
        for size in sizes:
            img_size.append(size.find('width').text)
            img_size.append(size.find('height').text)
            
        for obj in objects:
            defect_type=obj.find('name').text
            if defect_type=='Others':
                defect_type=0
            elif defect_type=='Porosity' or defect_type=='Porotisy':
                defect_type=1
            elif defect_type=='Slag' or defect_type=='slag inclusion':
                #print(defect_type)
                defect_type=2
                
            bndbox=obj.find('bndbox')
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)
            
            bboxes.append([xmin,ymin,xmax,ymax])
            labels.append(defect_type)
        
        bndbox_list.append(bboxes)
        imgshape_list.append(img_size)
        label_list.append(labels)

주의 : Label/Slag 폴더에 (no_object) 라고 되어 있는 파일명에서 (no_object 지우기)...

In [67]:
df['bndbox']=bndbox_list
df['img_shape']=imgshape_list
df['labels']=label_list

In [68]:
df.columns

Index(['dataset', 'image_number', 'image_name', 'bndbox', 'labels',
       'img_shape'],
      dtype='object')

In [71]:
df.head()

,dataset,image_number,image_name,bndbox,labels,img_shape
0,Others,1,6054-000130-1.jpg,"[[2519, 144, 2626, 188]]",[2],"[2980, 813]"
1,Others,2,6054-000130-1-2.jpg,"[[325, 346, 359, 374], [638, 293, 727, 332]]","[1, 2]","[2987, 823]"
2,Others,3,6054-000134-1-1.jpg,"[[469, 227, 584, 313]]",[2],"[1450, 826]"
3,Others,4,6054-000134-1-2.jpg,"[[963, 176, 1015, 237]]",[0],"[1486, 832]"
4,Others,5,6054-001180-A1-1.jpg,"[[950, 267, 1037, 301], [153, 301, 747, 341]]","[2, 2]","[3039, 859]"


In [72]:
df.to_csv('./data/annotations/annotations_total_merged.csv',index=False)

train_valid_test split

In [73]:
merged_df=pd.read_csv('./data/annotations/annotations_total_merged.csv')

In [74]:
merged_df.shape

(2274, 6)

In [75]:
# 8:1:1 로 나누기

# 1819, 227,228

train_df=merged_df.sample(n=1819)
merged_df=merged_df.drop(train_df.index,axis=0,inplace=False)

train_df.shape,merged_df.shape

((1819, 6), (455, 6))

In [76]:
train_df = train_df.reset_index(drop=True)
merged_df = merged_df.reset_index(drop=True)

In [77]:
train_df.to_csv('./data/annotations/train_total.csv',index=False)

In [78]:
# valid test split
valid_df=merged_df.sample(n=227)
merged_df=merged_df.drop(valid_df.index,axis=0,inplace=False)

valid_df.shape,merged_df.shape

((227, 6), (228, 6))

In [79]:
valid_df.to_csv('./data/annotations/valid_total.csv',index=False)
merged_df.to_csv('./data/annotations/test_total.csv',index=False)